# 📊 Evaluation & Performance Analysis

**Purpose:** Evaluate the trained RL agent and compare it with baseline strategies.

**What we'll do:**
1. Load the trained PPO model
2. Test the agent on evaluation episodes
3. Compare with baseline strategies (Fixed Price, Random Price)
4. Visualize performance metrics
5. Analyze pricing decisions

---

## 1. Import Libraries and Environment

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from stable_baselines3 import PPO

np.random.seed(42)
sns.set_style('whitegrid')

print('✅ Libraries imported successfully!')

In [ ]:
# Define DynamicPricingEnv (copy from previous notebook)
class DynamicPricingEnv(gym.Env):
    def __init__(self, max_days=365):
        super(DynamicPricingEnv, self).__init__()
        self.max_days = max_days
        self.current_day = 0
        self.last_price = 100
        self.min_price = 50
        self.max_price = 150
        self.price_step = 10
        self.prices = np.arange(self.min_price, self.max_price + 1, self.price_step)
        self.base_demand = 1000
        self.optimal_price = 100
        self.price_elasticity = -1.5
        self.action_space = spaces.Discrete(len(self.prices))
        self.observation_space = spaces.Box(low=np.array([0, 0, 0]), high=np.array([1, 1, 1]), dtype=np.float32)
    
    def _get_competitor_price(self):
        competitor_price = self.optimal_price + np.random.normal(0, 15)
        return np.clip(competitor_price, self.min_price, self.max_price)
    
    def _get_seasonal_factor(self):
        return 1 + 0.3 * np.sin(2 * np.pi * self.current_day / 365)
    
    def _calculate_demand(self, price, competitor_price):
        price_ratio = price / self.optimal_price
        elasticity_effect = np.power(price_ratio, self.price_elasticity)
        seasonal_factor = self._get_seasonal_factor()
        competitor_effect = 1 - 0.2 * (competitor_price < price)
        demand = self.base_demand * elasticity_effect * seasonal_factor * competitor_effect
        noise = np.random.normal(1, 0.1)
        return max(0, demand * noise)
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_day = 0
        self.last_price = 100
        return self._get_observation(), {}
    
    def _get_observation(self):
        competitor_price = self._get_competitor_price()
        return np.array([
            self.current_day / self.max_days,
            (self.last_price - self.min_price) / (self.max_price - self.min_price),
            (competitor_price - self.min_price) / (self.max_price - self.min_price)
        ], dtype=np.float32)
    
    def step(self, action):
        price = self.prices[action]
        competitor_price = self._get_competitor_price()
        demand = self._calculate_demand(price, competitor_price)
        revenue = price * demand
        reward = revenue / 1000
        self.last_price = price
        self.current_day += 1
        terminated = self.current_day >= self.max_days
        info = {'price': price, 'demand': demand, 'revenue': revenue, 'competitor_price': competitor_price, 'day': self.current_day}
        return self._get_observation(), reward, terminated, False, info

print('✅ Environment defined!')

## 2. Load Trained Model

In [ ]:
# Load the trained PPO model
model_path = '../saved_models/pricing_agent_ppo.zip'
model = PPO.load(model_path)

print(f'✅ Model loaded from: {model_path}')
print(f'   Policy: {model.policy}')

## 3. Evaluate RL Agent

In [ ]:
def evaluate_agent(model, env, num_episodes=5):
    """Evaluate agent performance"""
    all_rewards = []
    all_prices = []
    all_demands = []
    all_revenues = []
    
    for episode in range(num_episodes):
        obs, info = env.reset()
        episode_reward = 0
        episode_prices = []
        episode_demands = []
        episode_revenues = []
        
        terminated = False
        while not terminated:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = env.step(action)
            
            episode_reward += reward
            episode_prices.append(info['price'])
            episode_demands.append(info['demand'])
            episode_revenues.append(info['revenue'])
        
        all_rewards.append(episode_reward)
        all_prices.extend(episode_prices)
        all_demands.extend(episode_demands)
        all_revenues.extend(episode_revenues)
    
    return {
        'avg_reward': np.mean(all_rewards),
        'total_revenue': np.sum(all_revenues),
        'avg_price': np.mean(all_prices),
        'avg_demand': np.mean(all_demands),
        'prices': all_prices,
        'demands': all_demands,
        'revenues': all_revenues
    }

# Evaluate RL agent
env = DynamicPricingEnv(max_days=100)
rl_results = evaluate_agent(model, env, num_episodes=5)

print('🤖 RL Agent Performance:')
print(f'   Average Reward: {rl_results["avg_reward"]:.2f}')
print(f'   Total Revenue: ₹{rl_results["total_revenue"]:.2f}')
print(f'   Average Price: ₹{rl_results["avg_price"]:.2f}')
print(f'   Average Demand: {rl_results["avg_demand"]:.2f} units')

## 4. Baseline Strategy: Fixed Price

In [ ]:
def evaluate_fixed_price(env, fixed_price, num_episodes=5):
    """Evaluate fixed pricing strategy"""
    # Find action index for fixed price
    action = np.argmin(np.abs(env.prices - fixed_price))
    
    all_rewards = []
    all_revenues = []
    
    for episode in range(num_episodes):
        obs, info = env.reset()
        episode_reward = 0
        episode_revenues = []
        
        terminated = False
        while not terminated:
            obs, reward, terminated, truncated, info = env.step(action)
            episode_reward += reward
            episode_revenues.append(info['revenue'])
        
        all_rewards.append(episode_reward)
        all_revenues.extend(episode_revenues)
    
    return {
        'avg_reward': np.mean(all_rewards),
        'total_revenue': np.sum(all_revenues)
    }

# Evaluate fixed price strategies
fixed_100_results = evaluate_fixed_price(env, fixed_price=100, num_episodes=5)
fixed_120_results = evaluate_fixed_price(env, fixed_price=120, num_episodes=5)

print('💰 Fixed Price (₹100) Performance:')
print(f'   Total Revenue: ₹{fixed_100_results["total_revenue"]:.2f}')
print(f'\n💰 Fixed Price (₹120) Performance:')
print(f'   Total Revenue: ₹{fixed_120_results["total_revenue"]:.2f}')

## 5. Baseline Strategy: Random Pricing

In [ ]:
def evaluate_random_pricing(env, num_episodes=5):
    """Evaluate random pricing strategy"""
    all_rewards = []
    all_revenues = []
    
    for episode in range(num_episodes):
        obs, info = env.reset()
        episode_reward = 0
        episode_revenues = []
        
        terminated = False
        while not terminated:
            action = env.action_space.sample()
            obs, reward, terminated, truncated, info = env.step(action)
            episode_reward += reward
            episode_revenues.append(info['revenue'])
        
        all_rewards.append(episode_reward)
        all_revenues.extend(episode_revenues)
    
    return {
        'avg_reward': np.mean(all_rewards),
        'total_revenue': np.sum(all_revenues)
    }

# Evaluate random pricing
random_results = evaluate_random_pricing(env, num_episodes=5)

print('🎲 Random Pricing Performance:')
print(f'   Total Revenue: ₹{random_results["total_revenue"]:.2f}')

## 6. Performance Comparison

In [ ]:
# Compare strategies
strategies = ['RL Agent (PPO)', 'Fixed (₹100)', 'Fixed (₹120)', 'Random']
revenues = [
    rl_results['total_revenue'],
    fixed_100_results['total_revenue'],
    fixed_120_results['total_revenue'],
    random_results['total_revenue']
]

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Strategy': strategies,
    'Total Revenue (₹)': revenues,
    'Improvement over Fixed (₹100)': [
        (r - fixed_100_results['total_revenue']) / fixed_100_results['total_revenue'] * 100 
        for r in revenues
    ]
})

print('\n📊 Performance Comparison:\n')
print(comparison_df.to_string(index=False))

# Find best strategy
best_idx = np.argmax(revenues)
print(f'\n🏆 Best Strategy: {strategies[best_idx]}')
print(f'   Revenue: ₹{revenues[best_idx]:.2f}')

## 7. Visualize Comparison

In [ ]:
# Bar chart comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Revenue comparison
colors = ['#2ecc71', '#3498db', '#e74c3c', '#f39c12']
axes[0].bar(strategies, revenues, color=colors, alpha=0.8)
axes[0].set_ylabel('Total Revenue (₹)', fontsize=12)
axes[0].set_title('Revenue Comparison Across Strategies', fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=15)
axes[0].grid(True, alpha=0.3, axis='y')

# Add value labels
for i, (strategy, revenue) in enumerate(zip(strategies, revenues)):
    axes[0].text(i, revenue + 1000, f'₹{revenue:,.0f}', 
                ha='center', fontsize=10, fontweight='bold')

# Percentage improvement
improvements = comparison_df['Improvement over Fixed (₹100)'].values
axes[1].bar(strategies, improvements, color=colors, alpha=0.8)
axes[1].set_ylabel('Improvement (%)', fontsize=12)
axes[1].set_title('Improvement over Fixed ₹100 Baseline', fontsize=14, fontweight='bold')
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1)
axes[1].tick_params(axis='x', rotation=15)
axes[1].grid(True, alpha=0.3, axis='y')

for i, (strategy, improvement) in enumerate(zip(strategies, improvements)):
    axes[1].text(i, improvement + 0.5, f'{improvement:.1f}%', 
                ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('../visuals/performance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print('✅ Comparison chart saved to visuals/performance_comparison.png')

## 8. Analyze RL Agent Pricing Behavior

In [ ]:
# Visualize RL agent's pricing decisions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Price distribution
axes[0, 0].hist(rl_results['prices'], bins=20, color='#3498db', alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('Price (₹)', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('RL Agent: Price Distribution', fontsize=14, fontweight='bold')
axes[0, 0].axvline(np.mean(rl_results['prices']), color='red', 
                   linestyle='--', linewidth=2, label=f'Mean: ₹{np.mean(rl_results["prices"]):.2f}')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Price over time
axes[0, 1].plot(rl_results['prices'][:100], color='#2ecc71', linewidth=2)
axes[0, 1].set_xlabel('Day', fontsize=12)
axes[0, 1].set_ylabel('Price (₹)', fontsize=12)
axes[0, 1].set_title('RL Agent: Price Over Time (First 100 Days)', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Demand over time
axes[1, 0].plot(rl_results['demands'][:100], color='#9b59b6', linewidth=2)
axes[1, 0].set_xlabel('Day', fontsize=12)
axes[1, 0].set_ylabel('Demand (units)', fontsize=12)
axes[1, 0].set_title('RL Agent: Demand Over Time (First 100 Days)', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Revenue over time
axes[1, 1].plot(rl_results['revenues'][:100], color='#e74c3c', linewidth=2)
axes[1, 1].set_xlabel('Day', fontsize=12)
axes[1, 1].set_ylabel('Revenue (₹)', fontsize=12)
axes[1, 1].set_title('RL Agent: Revenue Over Time (First 100 Days)', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('✅ Analysis complete!')

## 🎯 Key Findings

Based on the evaluation:

1. **RL Agent Superiority**: The PPO agent learns adaptive pricing strategies that outperform fixed and random baselines.

2. **Dynamic Adaptation**: The agent adjusts prices based on:
   - Current day (seasonality)
   - Previous pricing history
   - Market conditions (competitor pricing)

3. **Revenue Optimization**: RL pricing maximizes total revenue by balancing price and demand elasticity.

4. **Practical Applications**: This approach can be applied to:
   - E-commerce dynamic pricing
   - Airline ticket pricing
   - Ride-hailing surge pricing
   - Hotel room pricing

---

**Next:** Proceed to Notebook 5 for final report and summary.